<a href="https://colab.research.google.com/github/meghrajeev/llm_hw3/blob/master/11667_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/meghrajeev/llm_hw3.git


fatal: destination path 'llm_hw3' already exists and is not an empty directory.


In [3]:
!git config --global user.email "mrajeev@andrew.cmu.edu"
!git config --global user.name "meghrajeev"

In [4]:
!pip install datasets transformers

In [5]:
from datasets import load_dataset

In [6]:
data_files = {
        "train": "en.noclean/c4-train.0000[0-1]-of-07168.json.gz",
    }
dataset = load_dataset("allenai/c4", data_files=data_files, split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [7]:
dataset

Dataset({
    features: ['text', 'timestamp', 'url'],
    num_rows: 296821
})

In [ ]:
dataset["text"][0]

In [ ]:
# Word Count

word_count = 0
for record in dataset:
    text = record['text']
    words = text.split()
    word_count += len(words)

print(f"Total number of words in the dataset: {word_count}")

In [8]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [9]:
# tokenization
def tokenize_function(examples):
    return tokenizer(examples["text"], add_special_tokens=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [11]:
tokenized_dataset

Dataset({
    features: ['text', 'timestamp', 'url', 'input_ids', 'attention_mask'],
    num_rows: 296821
})

In [ ]:
# histogram of document lengths
# Calculate the lengths of the tokenized documents
token_lengths = [len(input_id) for input_id in tokenized_dataset['input_ids']]
print("Total number of tokens:", sum(token_lengths))


In [ ]:
import matplotlib.pyplot as plt
# Plotting the histogram
plt.hist(token_lengths, bins=50, color='blue', alpha=0.7)
plt.title('Histogram of Document Sizes (Number of Tokens)')
plt.xlabel('Number of Tokens')
plt.ylabel('Number of Documents')
plt.show()

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.6 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, AutoTokenizer
tokenizer = tokenizer = BertTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")